In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [3]:
%cd '/content/gdrive/My Drive/LDS8_K270_ONLINE_DoThiPhuong/LDS8_K270_DoThiPhuong_Cuoi_ky/Cau_4/'

/content/gdrive/My Drive/LDS8_K270_ONLINE_DoThiPhuong/LDS8_K270_DoThiPhuong_Cuoi_ky/Cau_4


In [4]:
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras import Sequential, Model 
from tensorflow.keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from tensorflow.keras import backend as k 
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping


# Large dataset ~8000 images: 2 class
- Strategy: 2
- VGG16: image(224,224)

In [5]:
img_width, img_height = 224, 224
train_data_dir = "dataset/training_set"
validation_data_dir = "dataset/test_set"

In [6]:
model = applications.VGG16(weights= 'imagenet',
                           include_top = False, #whether to include the 3 fully-connected layers at the top of the network
                           input_shape = (img_width, img_height, 3))

58900480/58889256 [==============================] - 1s 0us/step


In [7]:
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

# New dataset is big and similar to original dataset

In [8]:
for layer in model.layers[:6]:
  layer.trainable = False

In [9]:
# Adding custom layers - ANN
x = model.output
x = Flatten()(x)
x = Dense(1024, activation= 'relu')(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation= 'relu')(x)
predictions = Dense(2, activation='sigmoid')(x)

In [10]:
#creating the final model
model_final = Model(inputs = model.input, outputs = predictions)

In [11]:
model_final.compile(loss = 'categorical_crossentropy',
                    optimizer = optimizers.SGD(lr = 0.0001, momentum=0.9), # hoặc ='adam'
                    metrics = ['accuracy'])

In [12]:
# Initiate the train and test generators with data Augumentation 
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   horizontal_flip = True,
                                   fill_mode = "nearest",
                                   zoom_range = 0.3,
                                   width_shift_range = 0.3,
                                   height_shift_range=0.3,
                                   rotation_range=30)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [13]:
train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size = (img_height, 
                                                                   img_width),
                                                    batch_size = 32, 
                                                    class_mode = "categorical")

validation_generator = test_datagen.flow_from_directory(validation_data_dir,
                                                    target_size = (img_height, 
                                                                   img_width),
                                                    class_mode = "categorical")


Found 6400 images belonging to 2 classes.
Found 1600 images belonging to 2 classes.


In [14]:
# Save the model according to the conditions  
checkpoint = ModelCheckpoint("lion_tiger_vgg16_cp.h5", 
                             monitor='val_loss', 
                             verbose=1, 
                             save_best_only=True, 
                             save_weights_only=False, 
                             mode='auto', 
                             save_freq=1)
early = EarlyStopping(monitor='val_loss', 
                      min_delta=0.001, 
                      patience=10, 
                      verbose=1, 
                      mode='auto')

In [15]:
import datetime

In [16]:
t0 = datetime.datetime.now()
t0

datetime.datetime(2021, 11, 1, 2, 14, 25, 405365)

In [17]:
history = model_final.fit(train_generator,
                          steps_per_epoch = len(train_generator),
                          epochs = 100,
                          validation_data =validation_generator,
                          validation_steps = len(validation_generator),
                          verbose =1,
                          callbacks = [checkpoint, early])
# dùng optimizer = 'adam' thì dùng batch_size =32, ko dùng 2 dòng steps

Epoch 1/100
200/200 [==============================] - 1678s 8s/step - loss: 0.3809 - accuracy: 0.8155 - val_loss: 0.0662 - val_accuracy: 0.9787
Epoch 2/100
200/200 [==============================] - 190s 947ms/step - loss: 0.1088 - accuracy: 0.9617 - val_loss: 0.0595 - val_accuracy: 0.9844
Epoch 3/100
200/200 [==============================] - 190s 949ms/step - loss: 0.0760 - accuracy: 0.9731 - val_loss: 0.0471 - val_accuracy: 0.9875
Epoch 4/100
200/200 [==============================] - 191s 955ms/step - loss: 0.0572 - accuracy: 0.9803 - val_loss: 0.0515 - val_accuracy: 0.9869
Epoch 5/100
200/200 [==============================] - 194s 968ms/step - loss: 0.0559 - accuracy: 0.9816 - val_loss: 0.0401 - val_accuracy: 0.9906
Epoch 6/100
200/200 [==============================] - 193s 966ms/step - loss: 0.0465 - accuracy: 0.9837 - val_loss: 0.0466 - val_accuracy: 0.9875
Epoch 7/100
200/200 [==============================] - 191s 956ms/step - loss: 0.0407 - accuracy: 0.9855 - val_loss: 0.0

In [18]:
t1 = datetime.datetime.now()
t1-t0

datetime.timedelta(seconds=4839, microseconds=447071)

In [19]:
import pandas as pd

In [20]:
history_df = pd.DataFrame(history.history)
history_df.tail()

,loss,accuracy,val_loss,val_accuracy
12,0.026556,0.990937,0.051209,0.986875
13,0.017711,0.995625,0.045938,0.990000
14,0.023121,0.992031,0.042052,0.991250
15,0.020577,0.992969,0.044750,0.991250
16,0.023971,0.992500,0.036760,0.993125


In [21]:
from tensorflow.keras.models import load_model
model_final.save('lion_tiger_vgg16.h5')
print('Save!')

Save!


# Sử dụng model có sẵn cho kết quả tốt hơn model em tự build, thời gian chạy cũng ngắn hơn rất nhiều